In [30]:
# EXTRACTION OF THE DATA
lifetime_pathlist <- list.files(path = "data/lifetime", recursive = TRUE, full.names = TRUE)
data_lifetime <- c()

# Use lapply to read each .dat file into a list of data frames, skipping empty files
for (file in lifetime_pathlist) {
    if (file.info(file)$size > 0) { 
        data_file <- as.list(read.table(file))
        data_lifetime <- c(data_file, data_lifetime)
    }
}

data_lifetime <- unlist(data_lifetime)
data_lifetime <- as.list(data_lifetime)

precesion_pathlist <- list.files(path = "data/precesion", recursive = TRUE, full.names = TRUE)

for (file in precesion_pathlist) {
    if (file.info(file)$size > 0) { 
        data_file <- as.list(read.table(file))
        data_precesion <- c(data_file, data_lifetime)
    }
}

data_precesion <- unlist(data_precesion)
data_precesion <- as.list(data_precesion)

In [35]:
library(coda)
library(rjags)

model.lifetime <- "
model {
  for (i in 1:N) {
    t[i] <- p_0 + p_1 * t_0[i]
  }
  p_0 ~ dnorm(7.4, 4.4)
  p_1 ~ dnorm(14.9, 0.11)
}
"

# Prepare data for JAGS
data.lifetime <- list(t_0 = data_lifetime, N = length(data_lifetime))

# Initial values
init.lifetime <- list(
    p_0 = rnorm(1, 7.4, 4.4),
    p_1 = rnorm(1, 14.9, 0.11)
)

# Parameters to monitor
params.lifetime <- c("p_0", "p_1")

jags.lifetime <- jags.model(textConnection(model.lifetime), data = data.lifetime, inits = init.lifetime)
update(jags.lifetime, 1000)  # Burn-in

# Sample from the posterior distribution
samples.lifetime <- coda.samples(jags.lifetime, variable.names = params.lifetime, n.iter = 10000)

# Convert to MCMC object
mcmc.lifetime <- as.mcmc(samples.lifetime)

#print(summary(samples.lifetime))




Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 0
   Unobserved stochastic nodes: 2
   Total graph size: 39388

Initializing model


Iterations = 1001:11000
Thinning interval = 1 
Number of chains = 1 
Sample size per chain = 10000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

      Mean     SD Naive SE Time-series SE
p_0  7.404 0.4712 0.004712       0.004712
p_1 14.918 3.0313 0.030313       0.030313

2. Quantiles for each variable:

     2.5%    25%    50%    75%  97.5%
p_0 6.466  7.096  7.406  7.713  8.332
p_1 8.925 12.873 14.967 16.963 20.798

